In [22]:
import os
os.chdir("d:\\ML_DL_Projects\\DL_Project\\CNN_CLASSIFIER")
os.getcwd()

'd:\\ML_DL_Projects\\DL_Project\\CNN_CLASSIFIER'

In [23]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallBackConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath : Path

In [24]:
import os 
from deepClassifier.constants import *
from deepClassifier.utils import read_yaml, create_directories


class ConfigurationManager:
    def __init__(
        self,
        config_file_path= CONFIG_FILE_PATH,
        params_file_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        create_directories([self.config["artifacts_root"]])

    def get_prepare_call_back(self) -> PrepareCallBackConfig:
        config = self.config["prepare_callbacks"]

        model_ckpt_dir = os.path.dirname(config["checkpoint_model_filepath"])

        create_directories([
            Path(model_ckpt_dir),
            Path(config["tensorboard_root_log_dir"])
        ])


        prepare_Call_Back_config= PrepareCallBackConfig(
            root_dir= Path(config["root_dir"]),
            tensorboard_root_log_dir= Path(config["tensorboard_root_log_dir"]),
            checkpoint_model_filepath= Path(config["checkpoint_model_filepath"])
        )

        return prepare_Call_Back_config




In [25]:
import os
import tensorflow as tf
import time


class PrepareCallback:
    def __init__(self, config: PrepareCallBackConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )

    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]


In [26]:
try:
    config = ConfigurationManager()
    prepare_callback_config= config.get_prepare_call_back()
    prepare_callback= PrepareCallback(config=prepare_callback_config)
    callback_list=  prepare_callback.get_tb_ckpt_callbacks()

except Exception as e:
    raise e
